In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from scipy.stats import pearsonr



cnn_model = load_model("/Users/samarpanpoudel/Desktop/model 1/saved_models/model1.h5")
resnet_model = load_model("/Users/samarpanpoudel/Desktop/Transfer Learning/saved_models/resnet1.h5")
efficientnetbo_model=load_model("/Users/samarpanpoudel/Desktop/EfficientNet-BO/saved_models/efficientnet_bo.h5")



folder_path = r"/Users/samarpanpoudel/Desktop/test"

image_files = sorted(os.listdir(folder_path))[:12]

cnn_predictions = []
resnet_predictions = []
efficientnet_predictions=[]

for img_name in image_files:
    
    img_path = os.path.join(folder_path, img_name)
    
    # Load image (IMPORTANT: 256x256)
    img = image.load_img(img_path, target_size=(256, 256))
    img_array = image.img_to_array(img)
    #img_array1 = img_array / 255.0   
    img_array = np.expand_dims(img_array, axis=0)
    
    # CNN prediction
    cnn_pred = cnn_model.predict(img_array, verbose=0)
    cnn_prob_class1 = float(cnn_pred[0][1])
    cnn_predictions.append(cnn_prob_class1)
    
    # ResNet prediction
    resnet_pred = resnet_model.predict(img_array, verbose=0)
    resnet_prob_class1 = float(resnet_pred[0][1])
    resnet_predictions.append(resnet_prob_class1)

    # Efficientnet  prediction
    efficientnet_pred = efficientnetbo_model.predict(img_array, verbose=0)
    efficientnet_prob_class1 = float(efficientnet_pred[0][1])
    efficientnet_predictions.append(efficientnet_prob_class1)

# Convert to numpy
cnn_predictions = np.array(cnn_predictions)
resnet_predictions = np.array(resnet_predictions)
efficientnet_predictions=np.array(efficientnet_predictions)



correlation1, p_value1 = pearsonr(cnn_predictions, resnet_predictions)
correlation2, p_value2 = pearsonr(cnn_predictions, efficientnet_predictions)
correlation3, p_value3 = pearsonr(efficientnet_predictions, resnet_predictions)

print("CNN class-1 probabilities:\n", cnn_predictions)
print("\nResNet class-1 probabilities:\n", resnet_predictions)
print("\nEfficientNet class-1 probabilities:\n", efficientnet_predictions)
print("\nPearson Correlation1, cnn vs restnet:", correlation1)
print("P-value1:", p_value1)
print("\nPearson Correlation2, cnn vs efficient_net:", correlation2)
print("P-value2:", p_value2)
print("\nPearson Correlation3, efficient_net vs restnet:", correlation3)
print("P-value3:", p_value3)

CNN class-1 probabilities:
 [0.01784083 0.01872507 0.01874232 0.03732447 0.04102376 0.32590681
 0.99975103 0.99968445 0.99974281 0.99879646 0.80302519 0.99986565]

ResNet class-1 probabilities:
 [1.00685339e-02 1.23713585e-02 6.73966780e-02 2.14240327e-03
 1.19739363e-03 8.22069123e-05 1.00000000e+00 9.99986887e-01
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]

EfficientNet class-1 probabilities:
 [0.1003908  0.53548652 0.44392318 0.74190855 0.0278154  0.06896504
 1.         0.99811053 0.99965489 0.99999905 0.99990404 0.99997342]

Pearson Correlation1, cnn vs restnet: 0.9756302170012292
P-value1: 6.498110820144197e-08

Pearson Correlation2, cnn vs efficient_net: 0.8175900301650051
P-value2: 0.0011605998883009033

Pearson Correlation3, efficient_net vs restnet: 0.8756915842635431
P-value3: 0.00018905387721484697


In [2]:
import numpy as np
from tensorflow.keras.preprocessing import image
import os

folder_path = r"/Users/samarpanpoudel/Desktop/test"
image_files = sorted(os.listdir(folder_path))[:12]

img_list = []

for img_name in image_files:
    img_path = os.path.join(folder_path, img_name)
    
    img = image.load_img(img_path, target_size=(256, 256))
    img_array = image.img_to_array(img)
    #img_array = img_array / 255.0   # same preprocessing as training
    
    img_list.append(img_array)

# Convert to batch
img_array_batch = np.array(img_list)

print("Batch shape:", img_array_batch.shape)

Batch shape: (12, 256, 256, 3)


In [3]:
cnn_preds = cnn_model.predict(img_array_batch, verbose=0)
resnet_preds = resnet_model.predict(img_array_batch, verbose=0)
efficientnet_preds = efficientnetbo_model.predict(img_array_batch, verbose=0)

print("CNN predicted classes:")
print(np.argmax(cnn_preds, axis=1))

print("ResNet predicted classes:")
print(np.argmax(resnet_preds, axis=1))

print("EfficientNet  predicted classes:")
print(np.argmax(efficientnet_preds, axis=1))

CNN predicted classes:
[0 0 0 0 0 0 1 1 1 1 1 1]
ResNet predicted classes:
[0 0 0 0 0 0 1 1 1 1 1 1]
EfficientNet  predicted classes:
[0 1 0 1 0 0 1 1 1 1 1 1]


In [4]:
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
cosine_similarity([cnn_predictions],[resnet_predictions])

array([[0.98744891]])

In [6]:
cosine_similarity([cnn_predictions],[efficientnet_predictions])

array([[0.92220115]])

In [7]:
cosine_similarity([efficientnet_predictions],[resnet_predictions])

array([[0.92814208]])